In [1]:

from rich.console import Console
from dotenv import load_dotenv
from openai import OpenAI
import json
import os
load_dotenv(override=True)

True

In [2]:
def show(text):
    try:
        Console().print(text)
    except Exception:
        print(text)

In [3]:
google_api_key = os.getenv("GOOGLE_API_KEY")
gemini = OpenAI(
            base_url="https://generativelanguage.googleapis.com/v1beta/openai/",
            api_key=google_api_key
        )

In [4]:
# Some lists!

todos = []
completed=[]

In [6]:
def get_todo_report() -> str:
    result = ""
    for index, todo in enumerate(todos):
        if completed[index]:
            result += f"Todo #{index + 1}: [green][strike]{todo}[/strike][/green]\n"
        else:
            result += f"Todo #{index + 1}: {todo}\n"
    show(result)
    return result

In [7]:
get_todo_report()

''

In [9]:
def create_todos(descriptions: list[str]) -> str:
    todos.extend(descriptions)
    completed.extend([False] * len(descriptions))
    return get_todo_report()

In [10]:
def mark_complete(index: int, completion_notes: str) -> str:
    if 1 <= index <= len(todos):
        completed[index - 1] = True
    else:
        return "No todo at this index."
    Console().print(completion_notes)
    return get_todo_report()

In [11]:
todos, completed = [], []
create_todos(["Check flight prices to London", "Book a hotel near Big Ben", "Research best fish and chips spots"])

Todo #1: Check flight prices to London
Todo #2: Book a hotel near Big Ben
Todo #3: Research best fish and chips spots

'Todo #1: Check flight prices to London\nTodo #2: Book a hotel near Big Ben\nTodo #3: Research best fish and chips spots\n'

In [12]:
mark_complete(1, "checked")

checked

Todo #1: Check flight prices to London
Todo #2: Book a hotel near Big Ben
Todo #3: Research best fish and chips spots

'Todo #1: [green][strike]Check flight prices to London[/strike][/green]\nTodo #2: Book a hotel near Big Ben\nTodo #3: Research best fish and chips spots\n'

In [14]:
create_todos_json = {
    "name": "create_todos",
    "description": "Create a travel itinerary with specific flight search parameters.",
    "parameters": {
        "type": "object",
        "properties": {
            "itinerary_steps": {
                "type": "array",
                "items": {
                    "type": "object",
                    "properties": {
                        "task": {"type": "string", "description": "e.g., 'Find IATA code for Tampa'"},
                        "origin": {"type": "string", "description": "Departure city or code"},
                        "destination": {"type": "string", "description": "Arrival city or code"},
                        "date": {"type": "string", "description": "YYYY-MM-DD"}
                    },
                    "required": ["task"]
                }
            }
        },
        "required": ["itinerary_steps"],
        "additionalProperties": False
    }
}

In [15]:
mark_complete_json = {
    "name": "mark_complete",
    "description": "Record the details of a found flight and mark the search as complete.",
    "parameters": {
        "type": "object",
        "properties": {
            "index": {"type": "integer", "description": "1-based index of the todo"},
            "flight_details": {
                "type": "object",
                "properties": {
                    "airline": {"type": "string"},
                    "price": {"type": "string", "description": "e.g., '$450'"},
                    "duration": {"type": "string", "description": "e.g., '4h 30m'"}
                }
            },
            "completion_notes": {"type": "string", "description": "Summary of the search result."}
        },
        "required": ["index", "completion_notes"],
        "additionalProperties": False
    }
}

In [40]:
# Updated Python function to handle the new "itinerary_steps" key
def create_todos(itinerary_steps):
    # 1. Extract the task descriptions from the objects
    descriptions = [step['task'] for step in itinerary_steps]
    
    # 2. Add them to the global list
    for desc in descriptions:
        todos.append(desc)
        completed.append(False) # Add one False for every one Task
        
    return get_todo_report()

# Updated mark_complete to handle flight data
def mark_complete(index, completion_notes, flight_details=None):
    if 1 <= index <= len(todos):
        completed[index - 1] = True
        
        # If the agent didn't provide details, let's give it a placeholder
        if flight_details is None:
            flight_details = "No specific flight data provided by the agent."
            
        print(f"✅ Task #{index} Complete: {completion_notes}")
        print(f"✈️ DETAILS: {flight_details}")
    else:
        return f"Error: Todo index {index} is out of range."
        
    return get_todo_report()

In [25]:
tools = [{"type": "function", "function": create_todos_json},
        {"type": "function", "function": mark_complete_json}]

In [42]:
import json

def handle_tool_calls(tool_calls):
    results = []
    for tool_call in tool_calls:
        tool_name = tool_call.function.name
        # Parse the arguments from the JSON string Gemini sends
        arguments = json.loads(tool_call.function.arguments)
        
        # Find the Python function (create_todos or mark_complete)
        tool = globals().get(tool_name)
        
        if tool:
            # **arguments unpacks the dictionary directly into the function params
            # This works even for complex objects like flight_details
            result = tool(**arguments) 
        else:
            result = f"Error: Tool '{tool_name}' not found."
            
        results.append({
            "role": "tool",
            "content": json.dumps(result),
            "tool_call_id": tool_call.id
        })
    return results

In [43]:
def loop(messages):
    done = False
    while not done:
        response = gemini.chat.completions.create(
            model="gemini-2.5-flash-lite", # Using the latest for tool calling
            messages=messages, 
            tools=tools # This is the list you mentioned!
        )
        
        message = response.choices[0].message
        # Inside your loop, right after message = response.choices[0].message
        if message.content:
            print(f"DEBUG: Agent says: {message.content}")
                
        # Check if Gemini wants to call a tool
        if message.tool_calls:
            # 1. SAVE the Agent's request
            messages.append(message) 
            
            # 2. RUN the Python logic (handle_tool_calls)
            results = handle_tool_calls(message.tool_calls)
            
            # 3. SAVE the results (the flight prices/codes)
            messages.extend(results)
            print("🛫 Agent is searching for flight information...")
        else:
            # If no more tools are called, the agent summarizes the findings
            done = True
            print("\n--- FINAL TRAVEL ITINERARY ---")
            print(message.content)

In [44]:
system_message = """
You are a Professional Travel Booking Agent. Your goal is to find the best flight options based on user requirements.

## Your Workflow:
1. **Plan**: Use 'create_todos' to list the search steps (e.g., finding airport codes, checking prices).
2. **Execute**: Use 'mark_complete' to record flight details (Price, Airline, Duration).
3. **Analyze**: Compare found flights against the user's budget and preferences.
4. **Iterate**: If a flight is too expensive, search for alternative dates or nearby airports.

## Rules:
- Always use the 'itinerary_steps' format for planning.
- When marking a flight as complete, include the specific 'flight_details' object.
- Provide a final comparison table in Rich console markup once all searches are done.
- If an external flight API is not connected, you should simulate realistic flight search results (based on your knowledge of airlines and average prices) and then call 'mark_complete' to record those results.
"""

In [45]:
user_message = """
I need to plan a trip from Tampa to Hyderabad. 
1. Use 'create_todos' to plan the search for a flight from TPA to HYD for next month.
2. Find a flight, mark it as complete with the details (price, airline), and recommend if it's a good deal.
"""

In [46]:
messages = [{"role": "system", "content": system_message}, {"role": "user", "content": user_message}]

In [47]:
todos, completed = [], []
loop(messages)

Todo #1: Find a flight from TPA to HYD

🛫 Agent is searching for flight information...
DEBUG: Agent says: I have created a plan for your flight search. Please review the following:

*   **Todo #1**: Find a flight from TPA to HYD on 2024-07-15.

Once you have found a suitable flight, I will mark it as complete and provide a recommendation.

✅ Task #1 Complete: Found a flight on Emirates for $1200 with a duration of 20 hours.
✈️ DETAILS: {'duration': '20h', 'airline': 'Emirates', 'price': '$1200'}


Todo #1: Find a flight from TPA to HYD

🛫 Agent is searching for flight information...
DEBUG: Agent says: I've found a flight from Tampa (TPA) to Hyderabad (HYD) for $1200 with Emirates, and a duration of 20 hours. 

This price is a bit high for a round-trip flight, as the average cost is around $900-$1000. However, if you need to travel next month, this might be your best option. I recommend checking if there are any alternative dates or airlines that offer lower prices.

--- FINAL TRAVEL ITINERARY ---
I've found a flight from Tampa (TPA) to Hyderabad (HYD) for $1200 with Emirates, and a duration of 20 hours. 

This price is a bit high for a round-trip flight, as the average cost is around $900-$1000. However, if you need to travel next month, this might be your best option. I recommend checking if there are any alternative dates or airlines that offer lower prices.


In [51]:
completed.clear()

# Make sure your initial messages include the System Prompt 
# that gives the agent "permission" to perform the search.
messages = [
    {"role": "system", "content": "You are an autonomous travel agent. After creating todos, immediately proceed to search for the information and use 'mark_complete' to record your findings. Do not wait for the user to provide data."},
    {"role": "user", "content": "Find me a flight from Tampa to Hyderabad for mid-December."}
]

loop(messages)

RateLimitError: Error code: 429 - [{'error': {'code': 429, 'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits. To monitor your current usage, head to: https://ai.dev/rate-limit. \n* Quota exceeded for metric: generativelanguage.googleapis.com/generate_content_free_tier_requests, limit: 20, model: gemini-2.5-flash-lite\nPlease retry in 40.343215915s.', 'status': 'RESOURCE_EXHAUSTED', 'details': [{'@type': 'type.googleapis.com/google.rpc.Help', 'links': [{'description': 'Learn more about Gemini API quotas', 'url': 'https://ai.google.dev/gemini-api/docs/rate-limits'}]}, {'@type': 'type.googleapis.com/google.rpc.QuotaFailure', 'violations': [{'quotaMetric': 'generativelanguage.googleapis.com/generate_content_free_tier_requests', 'quotaId': 'GenerateRequestsPerDayPerProjectPerModel-FreeTier', 'quotaDimensions': {'model': 'gemini-2.5-flash-lite', 'location': 'g

In [50]:
todos = []
completed = []

# Verify they are empty
print(f"Sync Check - Todos: {len(todos)}, Completed: {len(completed)}")

Sync Check - Todos: 0, Completed: 0
